# Intro to SQL : Databases

- Pandas csv has downsides including: 
    + it's static 
    + loadtime is long
    + data is stored in memory (thus cannot work with data > 8G)
- SQL on the other hand:
    + loads the data on disk and not memory
    + is dynamic

## Querying
- SELECT : select without modification  
- FROM : dataset to query
- WHERE (col, operator, value) : conditional statement
    + has to come after SELECT and FROM
- LIMIT : limit number of returned results
- OR / AND : logical operators could be used in conditional statement
- ORDER BY 'col name' [ASC or DESC] : order column by ascending/descending order
    + nested order by orders by the first column and then the 2nd column (ex. last name , first name)


In [13]:
import sqlite3
import pandas

# Create a connection instance
df = pandas.read_csv('recent_grads.csv')
df.to_sql(recent_grads, conn, if_exists='append', index=False)

# Create a connection instance
#conn = sqlite3.connect('recent_grads.csv')

EmptyDataError: No columns to parse from file

**To execute a query, SQL query needs to be executed as a string. **

**This can be done using the Cursor class:**
- run a query against the database.
- parse the results from the database.
- convert the results to native Python objects.
- store the results within the Cursor instance as a local variable.

In [10]:
c = conn.cursor()
query = "select * from recent_grads;"
c.execute(query)
results = c.fetchall()
print(results[0:2])

OperationalError: no such table: recent_grads